# Unsloth-Mac Complete Demo

This notebook demonstrates the **complete Unsloth-Mac pipeline** - same API as Unsloth!

**Features:**
- ✅ Load ANY HuggingFace model
- ✅ LoRA configuration
- ✅ SFTTrainer (same as Unsloth)
- ✅ Real training
- ✅ Save methods (adapters, merged, GGUF)
- ✅ Works in Jupyter notebooks

**Just like Unsloth, but for Apple Silicon!**

## Installation

```bash
pip install unsloth-mac
```

Or from source:
```bash
cd /path/to/unsloth-mac
pip install -e .
```

## 1. Import - Same as Unsloth!

In [ ]:
# Unsloth (CUDA): from unsloth import FastLanguageModel, SFTTrainer
# Unsloth-Mac (MLX): Just change the import!

from unsloth_mlx import FastLanguageModel, SFTTrainer
from datasets import load_dataset

print("✓ Imports successful!")

## 2. Load Model - Same API as Unsloth

In [ ]:
# Load ANY HuggingFace model (not just mlx-community)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mlx-community/Llama-3.2-1B-Instruct-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
)

print("✓ Model loaded successfully!")
print(f"  Model: {model.model_name}")

## 3. Configure LoRA - Same API as Unsloth

In [ ]:
# Add LoRA adapters - exact same API as Unsloth
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # 0 is optimized
    bias="none",
)

print("✓ LoRA configured!")
print(f"  Rank: {model.lora_config['r']}")
print(f"  Alpha: {model.lora_config['lora_alpha']}")

## 4. Prepare Dataset - Same as Unsloth

In [ ]:
# In real use: dataset = load_dataset("yahma/alpaca-cleaned", split="train")
# For demo, we'll create a small dataset

sample_data = [
    {
        "messages": [
            {"role": "user", "content": "What is Python?"},
            {"role": "assistant", "content": "Python is a high-level programming language."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "What is machine learning?"},
            {"role": "assistant", "content": "Machine learning is a subset of AI."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "What is 2+2?"},
            {"role": "assistant", "content": "2+2 equals 4."}
        ]
    },
]

print(f"✓ Dataset ready with {len(sample_data)} examples")

## 5. Train with SFTTrainer - Same API as Unsloth!

In [ ]:
# Initialize trainer - same API as Unsloth!
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=sample_data,
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    output_dir="./notebook_output",
    adapter_path="./notebook_adapters",
    iters=10,  # Small number for demo
)

print("✓ Trainer initialized!")
print(f"  Learning rate: {trainer.learning_rate}")
print(f"  Iterations: {trainer.iters}")

In [ ]:
# Train the model - this actually trains!
print("Starting training...")
trainer.train()
print("✓ Training complete!")

## 6. Inference - Same API as Unsloth

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Generate response
from mlx_lm import generate

prompt = "What is Python?"
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False
)

response = generate(
    model.model,
    tokenizer,
    prompt=formatted_prompt,
    max_tokens=100,
    verbose=False,
)

print(f"Q: {prompt}")
print(f"A: {response}")

## 7. Save Options - SAME AS UNSLOTH! 🎉

Three save methods, just like Unsloth:

### Option 1: Save LoRA Adapters Only (~100MB)

In [ ]:
# Save adapters only - small file size
# model.save_pretrained("lora_model")

print("Save adapters: model.save_pretrained('lora_model')")
print("✓ Same API as Unsloth!")

### Option 2: Save Merged Model (Base + Adapters)

In [ ]:
# Save merged model in HuggingFace format
# Anyone can use with transformers.AutoModel

# model.save_pretrained_merged(
#     "merged_16bit",
#     tokenizer,
#     save_method="merged_16bit"
# )

print("Save merged: model.save_pretrained_merged('merged_16bit', tokenizer)")
print("✓ Same API as Unsloth!")
print("✓ Others can use: transformers.AutoModel.from_pretrained('merged_16bit')")

### Option 3: Export to GGUF for llama.cpp/Ollama

In [ ]:
# Export to GGUF format
# model.save_pretrained_gguf(
#     "model",
#     tokenizer,
#     quantization_method="q4_k_m"
# )

print("Save GGUF: model.save_pretrained_gguf('model', tokenizer, quantization_method='q4_k_m')")
print("✓ Same API as Unsloth!")
print("✓ Use with llama.cpp, Ollama, GPT4All, etc.")

## Summary: API Comparison

### ✅ What's the SAME:

```python
# Both Unsloth and Unsloth-Mac:
model, tokenizer = FastLanguageModel.from_pretrained(...)
model = FastLanguageModel.get_peft_model(...)
trainer = SFTTrainer(model=model, train_dataset=dataset, ...)
trainer.train()
FastLanguageModel.for_inference(model)
model.save_pretrained("lora_model")
model.save_pretrained_merged("merged", tokenizer)
model.save_pretrained_gguf("model", tokenizer)
```

### ⚠️ What's DIFFERENT:

| Feature | Unsloth | Unsloth-Mac |
|---------|---------|-------------|
| Backend | CUDA/Triton | MLX |
| Trainer | TRL-based | MLX-based |
| Platform | NVIDIA GPUs | Apple Silicon |
| Import | `from unsloth import` | `from unsloth_mlx import` |

### 💡 The Point:

**Develop on Mac → Deploy on CUDA**

Just change the import line and your code works on both platforms!

## Next Steps

1. **Try with real datasets**: `load_dataset("yahma/alpaca-cleaned")`
2. **Train longer**: Increase `iters` or `num_train_epochs`
3. **Try larger models**: Llama 3.2 3B, 7B, etc.
4. **Share your model**: Upload to HuggingFace Hub
5. **Deploy**: Export to GGUF and use with Ollama

## Resources

- [Unsloth-Mac GitHub](https://github.com/yourusername/unsloth-mac)
- [Unsloth Documentation](https://docs.unsloth.ai)
- [MLX Documentation](https://ml-explore.github.io/mlx/)
- [Examples](../examples/)

---

**Just like Unsloth, but for Mac! 🚀**